# Tréning RL na vyvažovanie Cartpole

Tento notebook je súčasťou [kurikula AI pre začiatočníkov](http://aka.ms/ai-beginners). Bol inšpirovaný [oficiálnym tutoriálom PyTorch](https://pytorch.org/tutorials/intermediate/reinforcement_q_learning.html) a [touto implementáciou Cartpole v PyTorch](https://github.com/yc930401/Actor-Critic-pytorch).

V tomto príklade použijeme RL na natrénovanie modelu, ktorý dokáže vyvažovať tyč na vozíku, ktorý sa môže pohybovať doľava a doprava na horizontálnej osi. Na simuláciu tyče použijeme prostredie [OpenAI Gym](https://www.gymlibrary.ml/).

> **Note**: Kód tejto lekcie môžete spustiť lokálne (napr. vo Visual Studio Code), v takom prípade sa simulácia otvorí v novom okne. Pri spúšťaní kódu online môže byť potrebné vykonať niektoré úpravy kódu, ako je popísané [tu](https://towardsdatascience.com/rendering-openai-gym-envs-on-binder-and-google-colab-536f99391cc7).

Začneme tým, že sa uistíme, že Gym je nainštalovaný:


In [ ]:
import sys
!{sys.executable} -m pip install gym

Teraz vytvorme prostredie CartPole a pozrime sa, ako s ním pracovať. Prostredie má nasledujúce vlastnosti:

* **Akčný priestor** je množina možných akcií, ktoré môžeme vykonať v každom kroku simulácie  
* **Pozorovací priestor** je priestor pozorovaní, ktoré môžeme vykonať  


In [ ]:
import gym

env = gym.make("CartPole-v1")

print(f"Action space: {env.action_space}")
print(f"Observation space: {env.observation_space}")

Pozrime sa, ako funguje simulácia. Nasledujúca slučka spúšťa simuláciu, až kým `env.step` nevráti ukončovací príznak `done`. Náhodne vyberieme akcie pomocou `env.action_space.sample()`, čo znamená, že experiment pravdepodobne veľmi rýchlo zlyhá (prostredie CartPole sa ukončí, keď rýchlosť CartPole, jeho poloha alebo uhol prekročia určité limity).

> Simulácia sa otvorí v novom okne. Kód môžete spustiť viackrát a sledovať, ako sa správa.


In [ ]:
env.reset()

done = False
total_reward = 0
while not done:
   env.render()
   obs, rew, done, info = env.step(env.action_space.sample())
   total_reward += rew
   print(f"{obs} -> {rew}")
print(f"Total reward: {total_reward}")

Môžete si všimnúť, že pozorovania obsahujú 4 čísla. Sú to:
- Poloha vozíka
- Rýchlosť vozíka
- Uhol tyče
- Rýchlosť otáčania tyče

`rew` je odmena, ktorú dostávame pri každom kroku. V prostredí CartPole získavate 1 bod za každý simulačný krok a cieľom je maximalizovať celkovú odmenu, t. j. čas, počas ktorého dokáže CartPole udržať rovnováhu bez pádu.

Počas posilňovaného učenia je naším cieľom natrénovať **politiku** $\pi$, ktorá nám pre každý stav $s$ povie, akú akciu $a$ máme vykonať, teda v podstate $a = \pi(s)$.

Ak chcete pravdepodobnostné riešenie, môžete si politiku predstaviť ako návratnosť množiny pravdepodobností pre každú akciu, t. j. $\pi(a|s)$ by znamenalo pravdepodobnosť, že by sme mali vykonať akciu $a$ v stave $s$.

## Metóda Gradientu Politiky

V najjednoduchšom RL algoritme, nazývanom **Gradient Politiky**, budeme trénovať neurónovú sieť, aby predpovedala ďalšiu akciu.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch

num_inputs = 4
num_actions = 2

model = torch.nn.Sequential(
    torch.nn.Linear(num_inputs, 128, bias=False, dtype=torch.float32),
    torch.nn.ReLU(),
    torch.nn.Linear(128, num_actions, bias = False, dtype=torch.float32),
    torch.nn.Softmax(dim=1)
)

Budeme trénovať sieť vykonávaním mnohých experimentov a aktualizáciou našej siete po každom spustení. Definujme funkciu, ktorá vykoná experiment a vráti výsledky (tzv. **stopu**) - všetky stavy, akcie (a ich odporúčané pravdepodobnosti) a odmeny:


In [ ]:
def run_episode(max_steps_per_episode = 10000,render=False):    
    states, actions, probs, rewards = [],[],[],[]
    state = env.reset()
    for _ in range(max_steps_per_episode):
        if render:
            env.render()
        action_probs = model(torch.from_numpy(np.expand_dims(state,0)))[0]
        action = np.random.choice(num_actions, p=np.squeeze(action_probs.detach().numpy()))
        nstate, reward, done, info = env.step(action)
        if done:
            break
        states.append(state)
        actions.append(action)
        probs.append(action_probs.detach().numpy())
        rewards.append(reward)
        state = nstate
    return np.vstack(states), np.vstack(actions), np.vstack(probs), np.vstack(rewards)

Môžete spustiť jednu epizódu s netrénovanou sieťou a pozorovať, že celková odmena (t.j. dĺžka epizódy) je veľmi nízka:


In [ ]:
s, a, p, r = run_episode()
print(f"Total reward: {np.sum(r)}")

Jedným z náročných aspektov algoritmu gradientu politiky je použitie **diskontovaných odmien**. Myšlienka spočíva v tom, že vypočítame vektor celkových odmien v každom kroku hry a počas tohto procesu diskontujeme skoré odmeny pomocou nejakého koeficientu $gamma$. Výsledný vektor tiež normalizujeme, pretože ho použijeme ako váhu na ovplyvnenie nášho tréningu:


In [ ]:
eps = 0.0001

def discounted_rewards(rewards,gamma=0.99,normalize=True):
    ret = []
    s = 0
    for r in rewards[::-1]:
        s = r + gamma * s
        ret.insert(0, s)
    if normalize:
        ret = (ret-np.mean(ret))/(np.std(ret)+eps)
    return ret

Teraz sa pustíme do samotného tréningu! Spustíme 300 epizód, pričom v každej epizóde vykonáme nasledujúce kroky:

1. Spustíme experiment a zhromaždíme sledovanie.
1. Vypočítame rozdiel (`gradients`) medzi vykonanými akciami a predpokladanými pravdepodobnosťami. Čím menší je rozdiel, tým viac si môžeme byť istí, že sme vykonali správnu akciu.
1. Vypočítame diskontované odmeny a vynásobíme gradienty diskontovanými odmenami - tým zabezpečíme, že kroky s vyššími odmenami budú mať väčší vplyv na konečný výsledok než tie s nižšími odmenami.
1. Očakávané cieľové akcie pre našu neurónovú sieť budú čiastočne odvodené z predpokladaných pravdepodobností počas behu a čiastočne z vypočítaných gradientov. Použijeme parameter `alpha` na určenie, do akej miery sa berú do úvahy gradienty a odmeny - toto sa nazýva *rýchlosť učenia* algoritmu posilneného učenia.
1. Nakoniec trénujeme našu sieť na stavoch a očakávaných akciách a proces opakujeme.


In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

def train_on_batch(x, y):
    x = torch.from_numpy(x)
    y = torch.from_numpy(y)
    optimizer.zero_grad()
    predictions = model(x)
    loss = -torch.mean(torch.log(predictions) * y)
    loss.backward()
    optimizer.step()
    return loss

In [ ]:
alpha = 1e-4

history = []
for epoch in range(300):
    states, actions, probs, rewards = run_episode()
    one_hot_actions = np.eye(2)[actions.T][0]
    gradients = one_hot_actions-probs
    dr = discounted_rewards(rewards)
    gradients *= dr
    target = alpha*np.vstack([gradients])+probs
    train_on_batch(states,target)
    history.append(np.sum(rewards))
    if epoch%100==0:
        print(f"{epoch} -> {np.sum(rewards)}")

plt.plot(history)

Teraz spustime epizódu s vykresľovaním, aby sme videli výsledok:


In [ ]:
_ = run_episode(render=True)

Dúfam, že vidíte, že tyč sa teraz dokáže celkom dobre vyvážiť!

## Model Actor-Critic

Model Actor-Critic je ďalším vývojom gradientov politiky, v ktorom vytvárame neurónovú sieť na učenie sa politiky aj odhadovaných odmien. Sieť bude mať dva výstupy (alebo ju môžete vnímať ako dve samostatné siete):
* **Actor** odporučí akciu, ktorú treba vykonať, tým, že nám poskytne pravdepodobnostné rozdelenie stavov, podobne ako v modeli gradientu politiky.
* **Critic** odhadne, aké by mohli byť odmeny z týchto akcií. Vracia celkové odhadované odmeny v budúcnosti pre daný stav.

Definujme takýto model:


In [ ]:
from itertools import count
import torch.nn.functional as F

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
env = gym.make("CartPole-v1")

state_size = env.observation_space.shape[0]
action_size = env.action_space.n
lr = 0.0001

class Actor(torch.nn.Module):
    def __init__(self, state_size, action_size):
        super(Actor, self).__init__()
        self.state_size = state_size
        self.action_size = action_size
        self.linear1 = torch.nn.Linear(self.state_size, 128)
        self.linear2 = torch.nn.Linear(128, 256)
        self.linear3 = torch.nn.Linear(256, self.action_size)

    def forward(self, state):
        output = F.relu(self.linear1(state))
        output = F.relu(self.linear2(output))
        output = self.linear3(output)
        distribution = torch.distributions.Categorical(F.softmax(output, dim=-1))
        return distribution


class Critic(torch.nn.Module):
    def __init__(self, state_size, action_size):
        super(Critic, self).__init__()
        self.state_size = state_size
        self.action_size = action_size
        self.linear1 = torch.nn.Linear(self.state_size, 128)
        self.linear2 = torch.nn.Linear(128, 256)
        self.linear3 = torch.nn.Linear(256, 1)

    def forward(self, state):
        output = F.relu(self.linear1(state))
        output = F.relu(self.linear2(output))
        value = self.linear3(output)
        return value

Budeme musieť mierne upraviť naše funkcie `discounted_rewards` a `run_episode`:


In [ ]:
def discounted_rewards(next_value, rewards, masks, gamma=0.99):
    R = next_value
    returns = []
    for step in reversed(range(len(rewards))):
        R = rewards[step] + gamma * R * masks[step]
        returns.insert(0, R)
    return returns

def run_episode(actor, critic, n_iters):
    optimizerA = torch.optim.Adam(actor.parameters())
    optimizerC = torch.optim.Adam(critic.parameters())
    for iter in range(n_iters):
        state = env.reset()
        log_probs = []
        values = []
        rewards = []
        masks = []
        entropy = 0
        env.reset()

        for i in count():
            env.render()
            state = torch.FloatTensor(state).to(device)
            dist, value = actor(state), critic(state)

            action = dist.sample()
            next_state, reward, done, _ = env.step(action.cpu().numpy())

            log_prob = dist.log_prob(action).unsqueeze(0)
            entropy += dist.entropy().mean()

            log_probs.append(log_prob)
            values.append(value)
            rewards.append(torch.tensor([reward], dtype=torch.float, device=device))
            masks.append(torch.tensor([1-done], dtype=torch.float, device=device))

            state = next_state

            if done:
                print('Iteration: {}, Score: {}'.format(iter, i))
                break


        next_state = torch.FloatTensor(next_state).to(device)
        next_value = critic(next_state)
        returns = discounted_rewards(next_value, rewards, masks)

        log_probs = torch.cat(log_probs)
        returns = torch.cat(returns).detach()
        values = torch.cat(values)

        advantage = returns - values

        actor_loss = -(log_probs * advantage.detach()).mean()
        critic_loss = advantage.pow(2).mean()

        optimizerA.zero_grad()
        optimizerC.zero_grad()
        actor_loss.backward()
        critic_loss.backward()
        optimizerA.step()
        optimizerC.step()


Teraz spustíme hlavnú tréningovú slučku. Použijeme manuálny proces tréningu siete výpočtom správnych stratových funkcií a aktualizáciou parametrov siete:


In [ ]:

actor = Actor(state_size, action_size).to(device)
critic = Critic(state_size, action_size).to(device)
run_episode(actor, critic, n_iters=100)

In [ ]:
env.close()

## Zhrnutie

V tejto ukážke sme videli dva algoritmy posilneného učenia: jednoduchý policy gradient a sofistikovanejší actor-critic. Môžete si všimnúť, že tieto algoritmy pracujú s abstraktnými pojmami ako stav, akcia a odmena – preto ich možno aplikovať na veľmi odlišné prostredia.

Posilnené učenie nám umožňuje naučiť sa najlepšiu stratégiu na vyriešenie problému len na základe sledovania konečnej odmeny. Skutočnosť, že nepotrebujeme označené dátové súbory, nám umožňuje opakovane simulovať a optimalizovať naše modely. Napriek tomu však v oblasti posilneného učenia existuje množstvo výziev, ktoré môžete objaviť, ak sa rozhodnete venovať viac pozornosti tejto zaujímavej oblasti umelej inteligencie.



---

**Upozornenie**:  
Tento dokument bol preložený pomocou služby AI prekladu [Co-op Translator](https://github.com/Azure/co-op-translator). Aj keď sa snažíme o presnosť, prosím, berte na vedomie, že automatizované preklady môžu obsahovať chyby alebo nepresnosti. Pôvodný dokument v jeho rodnom jazyku by mal byť považovaný za autoritatívny zdroj. Pre kritické informácie sa odporúča profesionálny ľudský preklad. Nie sme zodpovední za akékoľvek nedorozumenia alebo nesprávne interpretácie vyplývajúce z použitia tohto prekladu.
